In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ranro\\Documents\\ML_Projects\\end_to_end_ml_mlflow\\research'

In [3]:
os.chdir("../") # goes back one folder

In [4]:
%pwd

'c:\\Users\\ranro\\Documents\\ML_Projects\\end_to_end_ml_mlflow'

In [5]:
#######################################################
# config_entity.py:
# Used to frame a data ingestion configs as an object
#######################################################

from dataclasses import dataclass # used for defining a class with variables & types
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_dirs

In [7]:
class ConfigurationManager:
    """
    Class to represent a configuration managder for initialize
    configurations from config.yaml.
    """
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_dirs([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Get data ingestion configurations.

        Args:
            None
        
        Returns: 
            DataIngestionConfig object that contains the configs from config.yaml

        """
        config = self.config.data_ingestion

        create_dirs([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
    

In [8]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [9]:
class DataIngestion:
    """
    Class that deals with data ingestion
    """
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    

    def download_file(self):
        """
        Downloads the data.

        Args:
            None
        
        Returns: 
            None

        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded successfully! additional info: {headers}")
        else:
            logger.info(f"File alrady exists, with size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        Extracts a zip file into a directory.

        Args:
            None
        
        Returns: 
            DataIngestionConfig object that contains the configs from config.yaml

        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_f:
            zip_f.extractall(unzip_path)

In [25]:
# Defines the pipeline for downloading and extracting the data
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-18 15:09:58,522: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-18 15:09:58,526: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-18 15:09:58,527: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-18 15:09:58,529: INFO: common: Folder artifacts was created successfully]
[2023-11-18 15:09:58,530: INFO: common: Folder artifacts/data_ingestion was created successfully]
[2023-11-18 15:09:58,531: INFO: 3345463528: File alrady exists, with size: The file artifacts\data_ingestion\data.zip has a size of ~23 KB]
